<font face="Calibri (Body)" color="blue" size="5">Gorilla Data Engineer Assessment</font>
<font face="Calibri (Body)" color="black" size="3">Answer 1</font>

Use pandas to calculate a transportation distribution charge for four gas meters in
the United Kingdom. Save your code in a Jupyter notebook and upload to a public
repo on Github (or any other platform of your choice). While solving this exercise,
focus on efficiency - i.e., use vectorised operations and avoid loops! All

*Transportation distribution charges are levied by gas distribution companies for the use of their
lower pressure pipelines; they cover the cost of physically transporting the gas through the
pipeline. This rate is determined depending on a meter's exit zone (gas network region) and its
estimated annual quantity (AQ); and it changes over time.
The daily charge is calculated by finding the correct rate for each meter and day in the
forecast and multiplying this rate (in p/kWh) with the day's forecast (in kWh).
Calculate the total cost per meter by summing its daily charges for the full forecast
period and converting to Pounds (1p = 0.01£).
Calculate the total consumption per meter by summing its daily consumption
forecast for the full period.
Your result should be a DataFrame*


The daily charge is calculated by finding the correct rate for each meter and day in the
forecast and multiplying this rate (in p/kWh) with the day's forecast (in kWh).
Calculate the total cost per meter by summing its daily charges for the full forecast
period and converting to Pounds (1p = 0.01£).
Calculate the total consumption per meter by summing its daily consumption
forecast for the full period.

In [1]:
#pip install packages
from pandas.core.apply import frame_apply
import numpy as np
import pandas as pd

In [2]:
#generate dataframes for each table
df_meter = pd.read_csv("meter_table.csv")
df_forecast = pd.read_csv("forecast_table.csv")
df_rate = pd.read_csv("rate_table.csv")

check if the dataframes are correctly generated

In [3]:
print(df_meter.head(5))

   meter_id  aq_kwh exit_zone
0  14676236   28978       EA1
1  34509937   78324       SO1
2  50264822  265667       NT1
3  88357331  484399       SE2


In [4]:
print(df_forecast.head(5))

   meter_id                 date        kwh
0  14676236  2020-06-01 00:00:00  22.070768
1  14676236  2020-06-02 00:00:00  19.170720
2  14676236  2020-06-03 00:00:00  23.555111
3  14676236  2020-06-04 00:00:00  18.220712
4  14676236  2020-06-05 00:00:00  14.196134


In [5]:
print(df_rate.head(5))
# We can see that blank cell are converted to NaN 

                  date exit_zone  aq_min_kwh  aq_max_kwh  rate_p_per_kwh
0  2020-04-01 00:00:00       EA1           0     73200.0          0.2652
1  2020-04-01 00:00:00       EA1       73200    732000.0          0.1980
2  2020-04-01 00:00:00       EA1      732000         NaN          0.2875
3  2020-04-01 00:00:00       EA2           0     73200.0          0.2970
4  2020-04-01 00:00:00       EA2       73200    732000.0          0.1524


We will first merge the df_meter with the df_forcast to have the exitzone information for each meter.

In [6]:
df_forcast_meter = df_forecast.merge(df_meter, on='meter_id')

In [7]:
# Check the generated  table 
print(df_forcast_meter.head(5))

   meter_id                 date        kwh  aq_kwh exit_zone
0  14676236  2020-06-01 00:00:00  22.070768   28978       EA1
1  14676236  2020-06-02 00:00:00  19.170720   28978       EA1
2  14676236  2020-06-03 00:00:00  23.555111   28978       EA1
3  14676236  2020-06-04 00:00:00  18.220712   28978       EA1
4  14676236  2020-06-05 00:00:00  14.196134   28978       EA1


Now we want to merge the df_forcast_merged table with the df_rate table to end up with a table that contains all the information needed for calculating The daily charge. Here a problem will emerge as we have two columns with the similar name 'date'. In order to differenciate between the two colums after the merge, we will rename them now.
We will rename the following:

1.'date' -> 'rate_date' in the sheet df_rate

2.'date' -> 'forcast_date' in the sheet df_forcast

In [8]:
df_forcast_meter = df_forcast_meter.rename(columns={'date': 'forcast_date'})
print(df_forcast_meter.head(5))

   meter_id         forcast_date        kwh  aq_kwh exit_zone
0  14676236  2020-06-01 00:00:00  22.070768   28978       EA1
1  14676236  2020-06-02 00:00:00  19.170720   28978       EA1
2  14676236  2020-06-03 00:00:00  23.555111   28978       EA1
3  14676236  2020-06-04 00:00:00  18.220712   28978       EA1
4  14676236  2020-06-05 00:00:00  14.196134   28978       EA1


In [9]:
df_rate = df_rate.rename(columns={'date': 'rate_date'})
print(df_rate.head(5))

             rate_date exit_zone  aq_min_kwh  aq_max_kwh  rate_p_per_kwh
0  2020-04-01 00:00:00       EA1           0     73200.0          0.2652
1  2020-04-01 00:00:00       EA1       73200    732000.0          0.1980
2  2020-04-01 00:00:00       EA1      732000         NaN          0.2875
3  2020-04-01 00:00:00       EA2           0     73200.0          0.2970
4  2020-04-01 00:00:00       EA2       73200    732000.0          0.1524


Now we need to sum up the daily kwh for each period (e.g. 2020-04-01 - 2020-10-01 = period 1, 2020-10-01 - 2021-04-01 = period 2). We will generate a new df called df_halfyear_forcast. df_halfyear_forcast will contain the following fiels(columns) meter_id, exit_zone, forcast_p(period), kwh_sumpp(sum per period).

In [10]:
# First we need a list of all distinct rate_dates.
distinct_rate_dates = df_rate['rate_date'].unique()
distinct_rate_dates_df = pd.DataFrame({'rate_date': distinct_rate_dates})
print(distinct_rate_dates)

['2020-04-01 00:00:00' '2020-10-01 00:00:00' '2021-04-01 00:00:00'
 '2021-10-01 00:00:00' '2022-04-01 00:00:00' '2022-10-01 00:00:00'
 '2023-04-01 00:00:00' '2023-10-01 00:00:00' '2024-04-01 00:00:00'
 '2024-10-01 00:00:00']


In [11]:
# Define the list of dates and corresponding periods
periods = distinct_rate_dates
# Function that assigns the period based on the date
def assign_period(forcast_date):
    for i in range(len(periods)-1): # Substract one becasue we compare the date to second last and the last
        if periods[i] <= forcast_date < periods[i+1]: 
            return periods[i] 
    return periods[-1] # Add one becasue we want to return the name of the lower compare date 
                       # As we name te periods by thier start date

# Apply the function to the date column to create a new column with period values
df_forcast_meter['period'] = df_forcast_meter['forcast_date'].apply(assign_period)
df_forcast_meter = pd.DataFrame(df_forcast_meter)

# print the DataFrame
print(df_forcast_meter.head(5))

   meter_id         forcast_date        kwh  aq_kwh exit_zone  \
0  14676236  2020-06-01 00:00:00  22.070768   28978       EA1   
1  14676236  2020-06-02 00:00:00  19.170720   28978       EA1   
2  14676236  2020-06-03 00:00:00  23.555111   28978       EA1   
3  14676236  2020-06-04 00:00:00  18.220712   28978       EA1   
4  14676236  2020-06-05 00:00:00  14.196134   28978       EA1   

                period  
0  2020-04-01 00:00:00  
1  2020-04-01 00:00:00  
2  2020-04-01 00:00:00  
3  2020-04-01 00:00:00  
4  2020-04-01 00:00:00  


In [12]:
# print the DataFrame
print(df_forcast_meter.head(5))

   meter_id         forcast_date        kwh  aq_kwh exit_zone  \
0  14676236  2020-06-01 00:00:00  22.070768   28978       EA1   
1  14676236  2020-06-02 00:00:00  19.170720   28978       EA1   
2  14676236  2020-06-03 00:00:00  23.555111   28978       EA1   
3  14676236  2020-06-04 00:00:00  18.220712   28978       EA1   
4  14676236  2020-06-05 00:00:00  14.196134   28978       EA1   

                period  
0  2020-04-01 00:00:00  
1  2020-04-01 00:00:00  
2  2020-04-01 00:00:00  
3  2020-04-01 00:00:00  
4  2020-04-01 00:00:00  


In [13]:
df_forcast_meter_rate = pd.merge(df_forcast_meter, df_rate, left_on=['period', 'exit_zone'], right_on=['rate_date', 'exit_zone'])

In [14]:
# We look at the specifics of this tabel
print(df_forcast_meter_rate.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10236 entries, 0 to 10235
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meter_id        10236 non-null  int64  
 1   forcast_date    10236 non-null  object 
 2   kwh             10236 non-null  float64
 3   aq_kwh          10236 non-null  int64  
 4   exit_zone       10236 non-null  object 
 5   period          10236 non-null  object 
 6   rate_date       10236 non-null  object 
 7   aq_min_kwh      10236 non-null  int64  
 8   aq_max_kwh      6824 non-null   float64
 9   rate_p_per_kwh  10236 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 879.7+ KB
None


In [15]:
print(df_forcast_meter_rate.head(5))

   meter_id         forcast_date        kwh  aq_kwh exit_zone  \
0  14676236  2020-06-01 00:00:00  22.070768   28978       EA1   
1  14676236  2020-06-01 00:00:00  22.070768   28978       EA1   
2  14676236  2020-06-01 00:00:00  22.070768   28978       EA1   
3  14676236  2020-06-02 00:00:00  19.170720   28978       EA1   
4  14676236  2020-06-02 00:00:00  19.170720   28978       EA1   

                period            rate_date  aq_min_kwh  aq_max_kwh  \
0  2020-04-01 00:00:00  2020-04-01 00:00:00           0     73200.0   
1  2020-04-01 00:00:00  2020-04-01 00:00:00       73200    732000.0   
2  2020-04-01 00:00:00  2020-04-01 00:00:00      732000         NaN   
3  2020-04-01 00:00:00  2020-04-01 00:00:00           0     73200.0   
4  2020-04-01 00:00:00  2020-04-01 00:00:00       73200    732000.0   

   rate_p_per_kwh  
0          0.2652  
1          0.1980  
2          0.2875  
3          0.2652  
4          0.1980  


In order to remove the rows that give the wrong combination of 'aq_kwh' and 'aq_min_kwh',  'aq_max_kwh'  range we create a filter that only keeps the rows for which the 'aq_kwh' fals between the  'aq_min_kwh' and  'aq_max_kwh'.

In [16]:
filtered = (df_forcast_meter_rate['aq_kwh'] >= df_forcast_meter_rate['aq_min_kwh']) & (df_forcast_meter_rate['aq_kwh'] <= df_forcast_meter_rate['aq_max_kwh'])

# Here we apply the filter to the df
df_forcast_meter_rate = df_forcast_meter_rate[filtered] 

# Print the filtered DataFrame
print(df_forcast_meter_rate.head(5))

    meter_id         forcast_date        kwh  aq_kwh exit_zone  \
0   14676236  2020-06-01 00:00:00  22.070768   28978       EA1   
3   14676236  2020-06-02 00:00:00  19.170720   28978       EA1   
6   14676236  2020-06-03 00:00:00  23.555111   28978       EA1   
9   14676236  2020-06-04 00:00:00  18.220712   28978       EA1   
12  14676236  2020-06-05 00:00:00  14.196134   28978       EA1   

                 period            rate_date  aq_min_kwh  aq_max_kwh  \
0   2020-04-01 00:00:00  2020-04-01 00:00:00           0     73200.0   
3   2020-04-01 00:00:00  2020-04-01 00:00:00           0     73200.0   
6   2020-04-01 00:00:00  2020-04-01 00:00:00           0     73200.0   
9   2020-04-01 00:00:00  2020-04-01 00:00:00           0     73200.0   
12  2020-04-01 00:00:00  2020-04-01 00:00:00           0     73200.0   

    rate_p_per_kwh  
0           0.2652  
3           0.2652  
6           0.2652  
9           0.2652  
12          0.2652  


In [17]:
# Now we have a look at the table info to see if we have correctly reduced the amount of rows.
# In the previouse version we had 10236 rows. As the aq can only fall in to one of three categories after the filter.
# The amount of rows now should be 10236/3 which is 3412.
print(df_forcast_meter_rate.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3412 entries, 0 to 10234
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meter_id        3412 non-null   int64  
 1   forcast_date    3412 non-null   object 
 2   kwh             3412 non-null   float64
 3   aq_kwh          3412 non-null   int64  
 4   exit_zone       3412 non-null   object 
 5   period          3412 non-null   object 
 6   rate_date       3412 non-null   object 
 7   aq_min_kwh      3412 non-null   int64  
 8   aq_max_kwh      3412 non-null   float64
 9   rate_p_per_kwh  3412 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 293.2+ KB
None


We see that we have reduced the amount of rows in the table by filtering out the rows for which we do not need the rates.

In the next step we will multiply the kwh  with the rate_p_per_kwh and then devide by hundred to achieve the cost per day per meter in pounds.

In [18]:
# Calculate the values total cost per meter per day and convert to pound 
df_forcast_meter_rate['cost_day_meter_pound']= (df_forcast_meter_rate['kwh']*df_forcast_meter_rate['rate_p_per_kwh'])/100

In [19]:
print(df_forcast_meter_rate.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3412 entries, 0 to 10234
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   meter_id              3412 non-null   int64  
 1   forcast_date          3412 non-null   object 
 2   kwh                   3412 non-null   float64
 3   aq_kwh                3412 non-null   int64  
 4   exit_zone             3412 non-null   object 
 5   period                3412 non-null   object 
 6   rate_date             3412 non-null   object 
 7   aq_min_kwh            3412 non-null   int64  
 8   aq_max_kwh            3412 non-null   float64
 9   rate_p_per_kwh        3412 non-null   float64
 10  cost_day_meter_pound  3412 non-null   float64
dtypes: float64(4), int64(3), object(4)
memory usage: 319.9+ KB
None


Now we sum the cost_day_meter_pound per 'exit_zone'  per 'period'            

In [20]:
df_sum_period = df_forcast_meter_rate.groupby(['meter_id', 'period', ]).agg({'kwh':'sum', 'cost_day_meter_pound':'sum'})

In [21]:
print(df_sum_period.head(30))
# This table shows the totalconsumption and total cost per meter per period

                                        kwh  cost_day_meter_pound
meter_id period                                                  
14676236 2020-04-01 00:00:00    1769.354787              4.692329
         2020-10-01 00:00:00    8565.582872             25.439781
         2021-04-01 00:00:00    4717.664057             15.695668
         2021-10-01 00:00:00    8512.419037             31.717273
         2022-04-01 00:00:00    5375.856768             22.433450
         2022-10-01 00:00:00      37.122479              0.173510
34509937 2020-04-01 00:00:00    8217.361143             22.540222
         2020-10-01 00:00:00   24558.935300             75.445049
         2021-04-01 00:00:00    9900.663302             34.068182
         2021-10-01 00:00:00   22644.929723             87.273559
         2022-04-01 00:00:00   12915.243639             55.742192
         2022-10-01 00:00:00      86.866895              0.419915
50264822 2020-04-01 00:00:00   31661.043470             69.052736
         2

In [22]:
# We should rename the column names kwh to total_kwh_pp and the cost_day_meter_pound to total_cost_pp
df_sum_period = df_sum_period.rename(columns={'kwh': 'total_kwh_pp', 'cost_day_meter_pound': 'total_cost_pp'})

Now in order to calculate the total cost per meter and the total consumption per meter we perform another sumantion this time grouping by meter_id.

In [23]:
df_sum = df_sum_period .groupby(['meter_id' ]).agg({'total_kwh_pp':'sum', 'total_cost_pp':'sum'})

Now we need to resemble the result table given in the task pdf for the first question we need to rename the 'total_kwh_pp' and 'total_cost_pp' to 'total_kwh' and 'total_cost'.

In [24]:
df_sum = df_sum_period .groupby(['meter_id' ]).agg({'total_kwh_pp':'sum', 'total_cost_pp':'sum'})

In [25]:
df_sum = df_sum.rename(columns={ 'total_kwh_pp': 'Total_Estimated_Consumption_(kWh)', 'total_cost_pp': 'Total_Cost_(£)'})

In [26]:
# The printed table shows the final answer 
print(df_sum.head(10))

          Total_Estimated_Consumption_(kWh)  Total_Cost_(£)
meter_id                                                   
14676236                       28978.000000      100.152012
34509937                       78324.000001      275.489119
50264822                      265667.000001      731.244071
88357331                      484399.000002     1433.160171
